### Retrival and Streamlit app


In [1]:
# pip install langchain sentence-transformers faiss-cpu unstructured


In [2]:
import os
import streamlit as st
import pickle
import time
import langchain
from langchain import OpenAI
from langchain.chains import RetrievalQAWithSourcesChain
from langchain.chains.qa_with_sources.loading import load_qa_with_sources_chain
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.document_loaders import UnstructuredURLLoader
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import FAISS
from langchain.document_loaders import WebBaseLoader


In [3]:
import os
import pickle
from langchain.vectorstores import FAISS
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.document_loaders import UnstructuredURLLoader
from langchain.chains import RetrievalQAWithSourcesChain
from langchain.llms import HuggingFaceHub
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.document_loaders import WebBaseLoader
from dotenv import load_dotenv

# 🔹 Load environment variables (if needed)
load_dotenv()
hf_token = os.getenv("HUGGINGFACEHUB_API_TOKEN")
# 1️⃣ Load documents from web pages
urls = [
    "https://www.moneycontrol.com/news/business/banks/hdfc-bank-re-appoints-sanmoy-chakrabarti-as-chief-risk-officer-11259771.html",
    "https://www.moneycontrol.com/news/business/markets/market-corrects-post-rbi-ups-inflation-forecast-icrr-bet-on-these-top-10-rate-sensitive-stocks-ideas-11142611.html"
]
loader = WebBaseLoader(urls)
docs = loader.load()

# 2️⃣ Split documents into smaller chunks (improves retrieval)
splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)
split_docs = splitter.split_documents(docs)


In [4]:
# 3️⃣ Use an open-source embedding model (NO OpenAI)
embedding_model = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

# 4️⃣ Build FAISS vector index (local, no API calls)
vector_index = FAISS.from_documents(split_docs, embedding_model)


c:\Users\sebat\Proyects\equity_research_tool\venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [5]:
vector_index

In [6]:

# 5️⃣ Save vector index to disk (so you don't recompute every time)
with open("vector_index.pkl", "wb") as f:
    pickle.dump(vector_index, f)

print("✅ Vector index created and saved!")


✅ Vector index created and saved!


In [7]:

# 6️⃣ Load the vector index when needed
with open("vector_index.pkl", "rb") as f:
    vector_index = pickle.load(f)

# 7️⃣ Set up a local language model (LLM) to answer questions
llm = HuggingFaceHub(repo_id="google/flan-t5-small", model_kwargs={"temperature": 0.5})

# 8️⃣ Create the QA Chain
chain = RetrievalQAWithSourcesChain.from_llm(llm=llm, retriever=vector_index.as_retriever())



c:\Users\sebat\Proyects\equity_research_tool\venv\Lib\site-packages\huggingface_hub\utils\_deprecation.py:131: FutureWarning: 'InferenceApi' (from 'huggingface_hub.inference_api') is deprecated and will be removed from version '1.0'. `InferenceApi` client is deprecated in favor of the more feature-complete `InferenceClient`. Check out this guide to learn how to convert your script to use it: https://huggingface.co/docs/huggingface_hub/guides/inference#legacy-inferenceapi-client.
  warnings.warn(warning_message, FutureWarning)


In [8]:
# 9️⃣ Ask a question
query = "What is the price of Tiago iCNG?"
response = chain({"question": query}, return_only_outputs=True)

print("\n🔍 Answer:")
print(response)

c:\Users\sebat\Proyects\equity_research_tool\venv\Lib\site-packages\huggingface_hub\file_download.py:140: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\sebat\.cache\huggingface\hub\models--gpt2. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)
Token indices sequence length is longer than the specified maximum sequence length for this model (1782 > 1024). Ru


🔍 Answer:
{'answer': 'iCNG', 'sources': ''}
